In [ ]:
%pip install -qU openai langchain_community sentence_transformers

In [ ]:
from openai import OpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
import sentence_transformers
from datetime import datetime

# Point to the local server
client = OpenAI(base_url="http://192.168.4.72:1234/v1", api_key="lm-studio")

embedding_function=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_db = Chroma(persist_directory="./chromadb", embedding_function=embedding_function)

# Add text to today's file in the data folder
def add_to_file(text):
    with open(f"data/{datetime.now().strftime('%Y-%m-%d')}.txt", "a") as f:
        f.write(text + "\n")

history = [
    {"role": "system", "content": """You are a mental health chatbot. 
     You are here to help people with their mental health. You ask questions to help the user understand their feelings and emotions. 
     You can also provide resources and advice. You are not a therapist, but you can help people find the right resources. 
     Keep your questions short and simple. Ask open-ended questions to encourage the user to share more about their feelings.
     Do not spend too much time on one topic. Move on to another topic if you've already asked 4 questions on the same topic. This helps keep the conversation engaging.
     Avoid starting your sentences with a rephrasing of the user's input. Instead, ask questions or provide information that can help the user explore their feelings.
     Use your past conversations to better guide the conversation."""},
    {"role": "user", "content": "Hello. I'm ready to start our session"},
    {"role": "assistant", "content": "Great! Let's start. How are you feeling today?"},
]


while True:
    completion = client.chat.completions.create(
        # model="TheBloke/Llama-2-13B-chat-GGUF", # this field is currently unused
        model="lmstudio-community/Meta_Llama-3-8B-Instruct-GGUF", # this field is currently unused
        messages=history,
        temperature=0.0,
        stream=True,
    )

    new_message = {"role": "assistant", "content": ""}
    
    for chunk in completion:
        if chunk.choices[0].delta.content:
            print(chunk.choices[0].delta.content, end="", flush=True)
            new_message["content"] += chunk.choices[0].delta.content

    history.append(new_message)
    add_to_file("Assistant: " + new_message["content"])
    
    #Uncomment to see chat history
    import json
    gray_color = "\033[90m"
    reset_color = "\033[0m"
    print(f"{gray_color}\n{'-'*20} History dump {'-'*20}\n")
    print(json.dumps(history, indent=2))
    print(f"\n{'-'*55}\n{reset_color}")

    print()
    next_input = input("> ")
    add_to_file("Francis: " + next_input)
    search_results = vector_db.similarity_search(next_input, k=2)
    some_context = ""
    for result in search_results:
        some_context += result.page_content + "\n\n"
    history.append({"role": "user", "content": some_context + next_input})